# Battle of Neighborhoods Capstone Project

## Project Objective

‘A’ who is the Managing Director of a Company wants to expand his business and has selected 2 prime locations i.e. Toronto, Canada and New York City, USA. ‘A’ wants to understand the neighborhoods and local businesses in those cities so that he can understand how the cities will be in terms of diversity, food cuisines, living standards and the quality of life for the employees. This project will explore the neighborhoods of both the cities and determine which will be a best fit to their company to expand their business and which area to work upon.

## Clustering Neighborhoods of Toronto

First lets download the necessary packages

In [5]:
import numpy as np # This library is used to handle the data in a vectorized manner

import pandas as pd # This library is used for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # This library is used to convert an address into Geographical values i.e. latitude and longitude 

import requests # library to handle requests
from pandas.io.json import json_normalize # This will tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from sklearn package for clustering 
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # This library will help visualize the geographical values on a map

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [6]:
from bs4 import BeautifulSoup 
import xml # We will be scraping the data from Wikipedia 

Now lets scrape the data from Wikipedia using the Beautiful Soup Package and process the data

In [7]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [8]:
tab_post = soup.find('table')
fields = tab_post.find_all('td')

postcode = []
borough = []
neighbourhd = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhd.append(fields[i+2].text.strip())
        
df_t = pd.DataFrame(data=[postcode, borough, neighbourhd]).transpose()
df_t.columns = ['Postcode', 'Borough', 'Neighbourhood'] # Let our column names be as follows
df_t.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df_t.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough otherwise, Ignore the cells with a borough that is Not assigned.
df_t.drop(df_t[df_t.Borough == 'Not assigned'].index, inplace=True)

#Combine the neighborhoods that exist only in one postal code
df_t = df_t.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#df.head()

#Change unassigned Neighborhood to its Borough's name
df_t.loc[85,'Neighborhood'] = 'Queen\'s Park'

df_t.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


We can see the shape of the data is 103 rows and 3 columns

In [10]:
print (df_t.shape)

(103, 3)


### Now lets join the data with its latitude and longitude coordinates

We can find the links for the Coordinates at: http://cocl.us/Geospatial_data

In [11]:
#Now lets Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlon = pd.read_csv("http://cocl.us/Geospatial_data")
latlon.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlon.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
latlon.tail()

,PostalCode,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [13]:
latlon.shape

(103, 3)

In [14]:
#Joining the Lat and Lon dataframe to Neighborhoods dataframe
df_t.set_index("PostalCode")
latlon.set_index("PostalCode")
neigh=pd.merge(df_t, latlon)
neigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neigh['Borough'].unique()),
        neigh.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


We can see that Toronto has 10 boroughs and 103 neighborhoods

Now in order to cluster the neighborhoods lets find the coordinates of Toronto, Canada

In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


## Creating Map of Toronto with the Neighborhoods Superimposed

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neigh['Latitude'], neigh['Longitude'], neigh['Borough'], neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Try to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Lets try segmenting and clustering only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe to view the Neighborhood data <B> Scarborough </B> 

In [18]:
scarbor_data = neigh[neigh['Borough'] == 'Scarborough'].reset_index(drop=True)
scarbor_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
scarbor_data.shape

(17, 5)

In [20]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="scar_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [21]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarbor_data['Latitude'], scarbor_data['Longitude'], scarbor_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

## Exploring the Scarborough neighborhood in Toronto using Foursquare API

### Define Foursquare credentials and version

In [90]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: ""******"
CLIENT_SECRET:"********"


### Exploring the Neighborhoods

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [24]:
scarborough_venues = getNearbyVenues(names=scarbor_data['Neighborhood'],
                                   latitudes=scarbor_data['Latitude'],
                                   longitudes=scarbor_data['Longitude']
                                  )

Found 90 venues in 17 neighborhoods.


In [25]:
print(scarbor_data)


   PostalCode      Borough                                       Neighborhood  \
0         M1B  Scarborough                                     Malvern, Rouge   
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek   
2         M1E  Scarborough                  Guildwood, Morningside, West Hill   
3         M1G  Scarborough                                             Woburn   
4         M1H  Scarborough                                          Cedarbrae   
5         M1J  Scarborough                                Scarborough Village   
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park   
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge   
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West   
9         M1N  Scarborough                        Birch Cliff, Cliffside West   
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...   
11        M1R  Scarborough  

In [26]:
print(scarborough_venues.shape)
scarborough_venues.head()

(90, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [27]:
#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",10,10,10,10,10,10
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [28]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

There are 81 distinct venues in 56 categories.


### Analyzing each Neighborhood

In [29]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# adding the neighborhood column back to the dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# reordering all the columns
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Mean frequency occurrence of each category in all neighborhoods

In [30]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Accessories Store,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000,0.000000,0.000,0.000,0.000,0.000000,0.0,0.250000,0.0,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000,0.0,0.00,0.25,0.25,0.000,0.0,0.000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.250000,0.000,0.00,0.000,0.000000
1,"Birch Cliff, Cliffside West",0.000,0.000000,0.000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.25,0.000,0.000000,0.000000,0.25,0.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.000,0.25,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000,0.0,0.00,0.00,0.00,0.000,0.0,0.000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.250000,0.000,0.00,0.000,0.000000
2,Cedarbrae,0.000,0.000000,0.125,0.000,0.125,0.125000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.125,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000,0.000000,0.125,0.000000,0.125,0.00,0.000000,0.000000,0.125,0.0,0.0,0.000000,0.000,0.0,0.00,0.00,0.00,0.000,0.0,0.000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000,0.00,0.125,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000,0.000000,0.000,0.000,0.000,0.100000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000,0.100000,0.000000,0.00,0.0,0.0,0.0,0.000,0.100000,0.100,0.000000,0.100,0.00,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000,0.1,0.00,0.00,0.00,0.000,0.0,0.000,0.000,0.000000,0.1,0.0,0.000000,0.000000,0.200000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000,0.00,0.100,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.000,0.333333,0.000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000,0.0,0.00,0.00,0.00,0.000,0.0,0.000,0.000,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.333333,0.000,0.00,0.000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000,0.000000,0.000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000,0.166667,0.000000,0.00,0.0,0.0,0.0,0.000,0.000000,0.000,0.166667,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.333333,0.000,0.0,0.00,0.00,0.00,0.000,0.0,0.000,0.000,0.000000,0.0,0.0,0.166667,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000,0.00,0.000,0.166667
6,"Golden Mile, Clairlea, Oakridge",0.000,0.000000,0.000,0.000,0.200,0.000000,0.0,0.000000,0.2,0.1,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.000,0.000000,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.0,0.1,0.000000,0.100,0.0,0.00,0.00,0.00,0.000,0.1,0.000,0.000,0.000000,0.0,0.1,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000,0.10,0.000,0.000000
7,"Guildwood, Morningside, West Hill",0.000,0.000000,0.000,0.000,0.000,0.125000,0.0,0.125000,0.0,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.125,0.000000,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.125,0.0,0.00,0.00,0.00,0.125,0.0,0.125,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.125,0.125,0.000000,0.000,0.000000,0.000,0.00,0.000,0.000

### Top 10 Venues in each Neighborhood

In [31]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4          Accessories Store  0.00
5                       Park  0.00
6               Intersection  0.00
7         Italian Restaurant  0.00
8          Korean Restaurant  0.00
9             Medical Center  0.00


----Birch Cliff, Cliffside West----
                       venue  freq
0               Skating Rink  0.25
1      General Entertainment  0.25
2                       Café  0.25
3            College Stadium  0.25
4          Accessories Store  0.00
5                       Park  0.00
6         Italian Restaurant  0.00
7          Korean Restaurant  0.00
8  Latin American Restaurant  0.00
9                     Lounge  0.00


----Cedarbrae----
                  venue  freq
0  Caribbean Restaurant  0.12
1    Athletics & Sports  0.12
2       Thai Restaurant  0.12
3                Bakery  0.12
4   

### Sorting all venues in Descending Order

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Coffee Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Convenience Store,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Italian Restaurant,Bank,Noodle House,Fast Food Restaurant,Chinese Restaurant,Fried Chicken Joint,Gas Station,Thai Restaurant,Furniture / Home Store
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Furniture / Home Store,Pet Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Intersection,Bus Station,Soccer Field,Ice Cream Shop,Park,Metro Station,Convenience Store,Department Store
7,"Guildwood, Morningside, West Hill",Rental Car Location,Medical Center,Electronics Store,Bank,Restaurant,Mexican Restaurant,Intersection,Breakfast Spot,Department Store,College Stadium
8,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Discount Store,Department Store,Convenience Store,Coffee Shop,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store


In [34]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                                        Bar
2nd Most Common Venue                      Vietnamese Restaurant
3rd Most Common Venue                                Coffee Shop
4th Most Common Venue                              Grocery Store
5th Most Common Venue                      General Entertainment
6th Most Common Venue                                Gas Station
7th Most Common Venue                     Furniture / Home Store
8th Most Common Venue                        Fried Chicken Joint
9th Most Common Venue                       Fast Food Restaurant
10th Most Common Venue                         Electronics Store
Name: 11, dtype: object

### Clustering the Neighborhood using K- Means Clustering 

In [35]:
# set number of clusters to 3
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0], dtype=int32)

In [36]:
scarbor_data.drop(scarbor_data[scarbor_data.Neighborhood == 'Upper Rouge'].index, inplace = True)

scarborough_merged = scarbor_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Rental Car Location,Medical Center,Electronics Store,Bank,Restaurant,Mexican Restaurant,Intersection,Breakfast Spot,Department Store,College Stadium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Convenience Store,Grocery Store


In [37]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining each Cluster

Now we can see how each cluster differs in the categories 


In [38]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [39]:
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,0,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store
1,Scarborough,-79.160497,0,Bar,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,-79.188711,0,Rental Car Location,Medical Center,Electronics Store,Bank,Restaurant,Mexican Restaurant,Intersection,Breakfast Spot,Department Store,College Stadium
3,Scarborough,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,-79.239476,0,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Convenience Store,Grocery Store
5,Scarborough,-79.239476,0,Grocery Store,Playground,Business Service,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,-79.262029,0,Hobby Shop,Discount Store,Department Store,Convenience Store,Coffee Shop,Gym,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
7,Scarborough,-79.284577,0,Bakery,Bus Line,Intersection,Bus Station,Soccer Field,Ice Cream Shop,Park,Metro Station,Convenience Store,Department Store
8,Scarborough,-79.239476,0,American Restaurant,Skating Rink,Motel,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,-79.273304,0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Furniture / Home Store,Pet Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [40]:
scarborough_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,-79.264848,1,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


In [41]:
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.295849,2,Accessories Store,Middle Eastern Restaurant,Smoke Shop,Shopping Mall,Sandwich Place,Auto Garage,Bakery,Breakfast Spot,Electronics Store,Convenience Store


We can see from above how each cluster holds various categories

## Exploring the Newyork Neighborhoods

New York City has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them lets understand the data first.


The link to the dataset is:https://geo.nyu.edu/catalog/nyu_2451_34572
        
We can run a wget command and access the data. 

In [42]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [43]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

We can see how all the relevant data is in the features key, which is basically a list of the neighborhoods. Lets extract that data.

In [44]:

neighborhoods_data = newyork_data['features']

In [45]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [46]:

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [47]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [48]:

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [49]:
neighborhoods.shape

(306, 4)

In [50]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Getting Geographical Coordinates of Newyork

In [51]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'ny-explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [52]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Lets cluster only the neighborhoods in Manhattan

In [53]:
# Lets take queens borough
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [55]:

geolocator = Nominatim(user_agent = 'qu-app')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7127281, -74.0060152.


In [56]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

### Lets explore the Long Island City neighborhood in Queens, NY

In [57]:
queens_data.loc[10, 'Neighborhood']

'Long Island City'

In [58]:
#Long Island City Latitude and Longitude values

neighborhood_latitude = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


### Top 100 venues in Long Island City neighborhood within a radius of 500 meters

In [59]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=3XPESCJQWKLWGSXG5UEMY4RDJQQCUCWOZZU0LQDUHFWB1CPC&client_secret=MKSQ1DC5ZSVR2BRXQXV4CNOE3MW2P2XFQ5ROVMHTK2PVF4L4&v=20200725&ll=40.75021734610528,-73.93920223915505&radius=500&limit=100'

In [60]:
#Send the GET request
results = requests.get(url).json()

In [61]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [62]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Chip,Dessert Shop,40.750069,-73.940831
1,Etto Espresso Bar,Coffee Shop,40.748703,-73.940689
2,Hilton Garden Inn,Hotel,40.750216,-73.936886
3,Dutch Kills,Cocktail Bar,40.747830,-73.940108
4,Baker House Market,Convenience Store,40.752137,-73.939235


In [63]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

72 venues were returned by Foursquare.


### Analyzing each neighborhood in Queens

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [65]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude'])

Found 2097 venues in 81 neighborhoods.


In [66]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop


In [67]:

print(queens_venues.shape)
queens_venues.tail()

(2097, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2092,Queensbridge,40.756091,-73.945631,Queensbridge Basketball Courts,40.755060,-73.949103,Basketball Court
2093,Queensbridge,40.756091,-73.945631,The Ravel Hotel Gym,40.753787,-73.948815,Athletics & Sports
2094,Queensbridge,40.756091,-73.945631,Profundo Pool Club,40.753719,-73.948878,Hotel Pool
2095,Queensbridge,40.756091,-73.945631,Estate Garden And Grill,40.753700,-73.948841,Beer Garden
2096,Queensbridge,40.756091,-73.945631,Track 114,40.753008,-73.947833,Platform


In [70]:
#Venues per Neighborhood
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,21,21,21,21,21,21
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,19,19,19,19,19,19
Bay Terrace,40,40,40,40,40,40
Bayside,72,72,72,72,72,72
Bayswater,1,1,1,1,1,1
Beechhurst,13,13,13,13,13,13
Bellaire,12,12,12,12,12,12


In [71]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))

There are 1728 distinct venues in 276 categories.


### Processing the data

In [72]:
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = queens_onehot['Neighborhood']
queens_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
queens_onehot.insert(0, 'Neighborhood', neighbor)

queens_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weig

### Frequency Occurance of each Venue

In [73]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weig

### Each neighborhood with the top 10 venues

In [74]:
num_top_venues = 10

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
             venue  freq
0        Surf Spot  0.19
1   Sandwich Place  0.10
2    Metro Station  0.10
3      Coffee Shop  0.05
4    Burrito Place  0.05
5             Café  0.05
6       Board Shop  0.05
7  Bed & Breakfast  0.05
8            Beach  0.05
9  Thai Restaurant  0.05


----Astoria----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1                        Bar  0.06
2           Greek Restaurant  0.05
3                 Hookah Bar  0.04
4   Mediterranean Restaurant  0.04
5         Seafood Restaurant  0.03
6                     Bakery  0.03
7                Pizza Place  0.03
8                 Food Truck  0.03
9          Indian Restaurant  0.03


----Astoria Heights----
                venue  freq
0  Italian Restaurant  0.15
1       Deli / Bodega  0.08
2              Hostel  0.08
3               Motel  0.08
4         Bus Station  0.08
5       Bowling Alley  0.08
6         Supermarket  0.08
7              Bakery  0.08
8         Pizza Place  0.08


In [75]:
#Sorting venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top Venues in each Neighborhoods

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Beach,Bed & Breakfast,Restaurant,Thai Restaurant,Coffee Shop,Board Shop,Donut Shop
1,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Indian Restaurant,Food Truck,Bakery,Pizza Place,Seafood Restaurant
2,Astoria Heights,Italian Restaurant,Plaza,Playground,Bowling Alley,Bus Station,Motel,Supermarket,Bakery,Burger Joint,Hostel
3,Auburndale,Italian Restaurant,Hookah Bar,Toy / Game Store,Pharmacy,Pet Store,Comic Shop,Discount Store,Noodle House,Fast Food Restaurant,Mobile Phone Shop
4,Bay Terrace,Clothing Store,Women's Store,Donut Shop,Lingerie Store,Shoe Store,Mobile Phone Shop,Kids Store,American Restaurant,Cosmetics Shop,Coffee Shop
5,Bayside,Bar,Pizza Place,Chinese Restaurant,American Restaurant,Sushi Restaurant,Indian Restaurant,Pub,Pharmacy,Donut Shop,Mexican Restaurant
6,Bayswater,Playground,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant
7,Beechhurst,Convenience Store,Italian Restaurant,Supermarket,Optical Shop,Donut Shop,Gym,Gym / Fitness Center,Park,Chinese Restaurant,Spa
8,Bellaire,IT Services,Italian Restaurant,Convenience Store,Coffee Shop,Chinese Restaurant,Moving Target,Bus Station,Greek Restaurant,Gym,Breakfast Spot
9,Belle Harbor,Beach,Deli / Bodega,Spa,Pub,Italian Restaurant,Boutique,Chinese Restaurant,Bakery,Bagel Shop,Donut Shop


In [77]:
neighborhoods_venues_sorted.iloc[47,]

Neighborhood                  Long Island City
1st Most Common Venue              Coffee Shop
2nd Most Common Venue                    Hotel
3rd Most Common Venue                      Bar
4th Most Common Venue              Pizza Place
5th Most Common Venue       Mexican Restaurant
6th Most Common Venue                     Café
7th Most Common Venue     Gym / Fitness Center
8th Most Common Venue               Donut Shop
9th Most Common Venue            Deli / Bodega
10th Most Common Venue         Bubble Tea Shop
Name: 47, dtype: object

### Performing K-Means Clustering on the Queens Borough

In [78]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Modified Dataframe which inclues cluster of each Neighborhood

In [79]:
queens_merged = queens_data

# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,0,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Indian Restaurant,Food Truck,Bakery,Pizza Place,Seafood Restaurant
1,Queens,Woodside,40.746349,-73.901842,0,Grocery Store,Bakery,Filipino Restaurant,Thai Restaurant,Latin American Restaurant,Bar,Pub,Donut Shop,American Restaurant,Pizza Place
2,Queens,Jackson Heights,40.751981,-73.882821,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Thai Restaurant,Mexican Restaurant,Grocery Store,Clothing Store,Empanada Restaurant
3,Queens,Elmhurst,40.744049,-73.881656,0,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Colombian Restaurant,Indonesian Restaurant,Bar,Malay Restaurant,Bakery
4,Queens,Howard Beach,40.654225,-73.838138,0,Bagel Shop,Italian Restaurant,Fast Food Restaurant,Clothing Store,Sandwich Place,Chinese Restaurant,Pharmacy,Deli / Bodega,Seafood Restaurant,Breakfast Spot


In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the Queens Clustered Neighborhood

Now, we can examine each cluster and determine the different venue categories that distinguish each cluster.

In [81]:
queens_cluster_0 = queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_1 = queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_2 = queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_3 = queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_4 = queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

### Understanding the various clusters and their values

In [82]:
queens_cluster_0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,0,Bar,Middle Eastern Restaurant,Greek Restaurant,Hookah Bar,Mediterranean Restaurant,Indian Restaurant,Food Truck,Bakery,Pizza Place,Seafood Restaurant
1,Woodside,0,Grocery Store,Bakery,Filipino Restaurant,Thai Restaurant,Latin American Restaurant,Bar,Pub,Donut Shop,American Restaurant,Pizza Place
2,Jackson Heights,0,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Thai Restaurant,Mexican Restaurant,Grocery Store,Clothing Store,Empanada Restaurant
3,Elmhurst,0,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Colombian Restaurant,Indonesian Restaurant,Bar,Malay Restaurant,Bakery
4,Howard Beach,0,Bagel Shop,Italian Restaurant,Fast Food Restaurant,Clothing Store,Sandwich Place,Chinese Restaurant,Pharmacy,Deli / Bodega,Seafood Restaurant,Breakfast Spot
5,Corona,0,Mexican Restaurant,Supermarket,South American Restaurant,Park,Sandwich Place,Chinese Restaurant,Basketball Court,Restaurant,Bakery,Donut Shop
7,Kew Gardens,0,Chinese Restaurant,Park,Indian Restaurant,Cosmetics Shop,Bar,Bank,Pizza Place,Donut Shop,Fried Chicken Joint,Spa
8,Richmond Hill,0,Pizza Place,Lounge,Latin American Restaurant,Bank,Discount Store,Sandwich Place,Supermarket,Clothing Store,Moving Target,Donut Shop
9,Flushing,0,Korean Restaurant,Chinese Restaurant,Hotpot Restaurant,Bubble Tea Shop,Bakery,Construction & Landscaping,Gym,Szechuan Restaurant,Karaoke Bar,Spa
10,Long Island City,0,Coffee Shop,Hotel,Bar,Pizza Place,Mexican Restaurant,Café,Gym / Fitness Center,Donut Shop,Deli / Bodega,Bubble Tea Shop


In [83]:

queens_cluster_1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Forest Hills,1,Gym / Fitness Center,Gym,Convenience Store,Thai Restaurant,Park,Yoga Studio,Pizza Place,Pharmacy,Optical Shop,Donut Shop


In [84]:
queens_cluster_2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Glendale,2,Brewery,Food & Drink Shop,Deli / Bodega,Pizza Place,Liquor Store,Arts & Crafts Store,Fast Food Restaurant,Eye Doctor,Falafel Restaurant,Farm


In [85]:
queens_cluster_3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Elmhurst,3,Donut Shop,Flower Shop,Chinese Restaurant,Ice Cream Shop,Hotel Bar,Lake,Rental Car Location,Gas Station,Coffee Shop,Supermarket
31,Jamaica Center,3,Mobile Phone Shop,Clothing Store,Pizza Place,Performing Arts Venue,Shoe Store,American Restaurant,Mexican Restaurant,Sandwich Place,Caribbean Restaurant,Department Store
52,Floral Park,3,Grocery Store,Pizza Place,Salon / Barbershop,Dosa Place,Basketball Court,Indian Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


In [86]:
queens_cluster_4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Utopia,4,Deli / Bodega,Spa,Automotive Shop,Pizza Place,Donut Shop,Bus Stop,South American Restaurant,Basketball Court,Bakery,Playground


In this project I have tried to differentiate the 2 locations to set up a new business and which business would bring more profits in a certain area. I have performed my analysis between the boroughs Scarborough in Toronto and Queens Borough in New York City. If we are select Scarborough we can see that a Grocery Store, Playground or a Business Service are the most common locations. Whereas, for Long Island City of Queens Borough a Coffee Shop, Hotel, Bar are the most common locaions. 

Now after deciding a specific location, when we need to decide on which business to expand on. If we select the most common venue we might end up having a lot of competition and we might need something special to stand out. If we select a category/ venue far from being the most common, we may not engage a huge range of audience. Selecting a category which would be beneficial on either way is suggested.

Thank you!